# 01 - Data Exploration

## Note
This notebook is kept for info only.

The raw data transformation steps shown in this notebook have now been abstracted out into the `energy_forecast` package.  Please see those implementations.  Run the data transformation pipeline from the commandline using:

```bash
make transform_data
```

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import energy_forecast
from energy_forecast.loaders import load_ods, load_ods_sheetnames
from energy_forecast.transform import clean_energy_data, clean_temp_data, clean_wind_data
from energy_forecast.transform import clean_sun_data, clean_rain_data
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from pathlib import Path

In [3]:
REPO_ROOT = Path.cwd().parent
DATA_RAW = REPO_ROOT / 'data' / 'raw'
DATA_INTERIM = REPO_ROOT / 'data' / 'interim'

ENERGY_PATH = DATA_RAW / 'Total_Energy_ODS.ods'
ENERGY_SHEETNAME = '1_2'

WEATHER_PATH = DATA_RAW / 'Weather_ODS.ods'
TEMP_SHEET = '7_1a'
WIND_SHEET = '7_2'
SUN_SHEET = '7_3'
RAIN_SHEET = '7_4'

In [4]:
# Run a helper function to load in the selected sheet as a dataframe
energy = load_ods(ENERGY_PATH, ENERGY_SHEETNAME)

In [5]:
# Run some data transformations to get the data into a useful tabular form
energy = clean_energy_data(energy)
energy

,total_energy,coal,petroleum,natural gas,bioenergy & waste,elec_nuclear,elec_renewable,elec_import
datetime,,,,,,,,
1995-01-01,21.03,4.32,6.16,8.90,0.14,1.51,NaN,NaN
1995-02-01,20.00,4.79,6.32,7.16,0.14,1.59,NaN,NaN
1995-03-01,22.54,5.62,6.96,7.91,0.14,1.91,NaN,NaN
1995-04-01,17.58,3.83,6.08,5.75,0.14,1.78,NaN,NaN
1995-05-01,16.43,3.72,6.23,4.79,0.14,1.55,NaN,NaN
...,...,...,...,...,...,...,...,...
2022-02-01,15.59,0.49,4.88,6.68,1.62,0.79,0.97,0.16
2022-03-01,15.49,0.54,5.01,6.54,1.62,0.97,0.67,0.14
2022-04-01,14.41,0.49,5.19,5.91,1.28,0.90,0.65,-0.01


In [6]:
energy.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 330 entries, 1995-01-01 to 2022-06-01
Data columns (total 8 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   total_energy       330 non-null    float64
 1   coal               330 non-null    float64
 2   petroleum          330 non-null    float64
 3   natural gas        330 non-null    float64
 4   bioenergy & waste  330 non-null    float64
 5   elec_nuclear       330 non-null    float64
 6   elec_renewable     294 non-null    float64
 7   elec_import        306 non-null    float64
dtypes: float64(8)
memory usage: 23.2 KB


# Weather data

In [7]:
temp = load_ods(WEATHER_PATH, TEMP_SHEET)
temp = clean_temp_data(temp)

wind = load_ods(WEATHER_PATH, WIND_SHEET)
wind = clean_wind_data(wind)

sun = load_ods(WEATHER_PATH, SUN_SHEET)
sun = clean_sun_data(sun)

rain = load_ods(WEATHER_PATH, RAIN_SHEET)
rain = clean_rain_data(rain)

In [8]:
weather = pd.concat([temp, wind, sun, rain], join='outer', axis=1)
weather

,temp,wind,sun,rain
datetime,,,,
1991-01-01,3.700000,NaN,NaN,NaN
1991-02-01,2.400000,NaN,NaN,NaN
1991-03-01,7.800000,NaN,NaN,NaN
1991-04-01,8.000000,NaN,NaN,NaN
1991-05-01,11.000000,NaN,NaN,NaN
...,...,...,...,...
2022-08-01,18.315515,6.522037,7.79751,74.999743
2022-09-01,NaN,NaN,NaN,NaN
2022-10-01,NaN,NaN,NaN,NaN


In [9]:
weather.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 384 entries, 1991-01-01 to 2022-12-01
Freq: MS
Data columns (total 4 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   temp    380 non-null    float64
 1   wind    260 non-null    float64
 2   sun     260 non-null    float64
 3   rain    260 non-null    float64
dtypes: float64(4)
memory usage: 15.0 KB


# Combine into one table

In [10]:
combined = pd.concat([energy, weather], axis=1)
combined

,total_energy,coal,petroleum,natural gas,bioenergy & waste,elec_nuclear,elec_renewable,elec_import,temp,wind,sun,rain
datetime,,,,,,,,,,,,
1991-01-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.700000,NaN,NaN,NaN
1991-02-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.400000,NaN,NaN,NaN
1991-03-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,7.800000,NaN,NaN,NaN
1991-04-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,8.000000,NaN,NaN,NaN
1991-05-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,11.000000,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
2022-08-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,18.315515,6.522037,7.79751,74.999743
2022-09-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2022-10-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


# Save prepared data to disk

In [11]:
# combined.to_csv(DATA_INTERIM / 'transformed_energy_weather.csv')